In [1]:
import time
import math
import pickle

import numpy as np
import scipy.linalg

from petsc4py import PETSc
from slepc4py import SLEPc

from ufl import *
from dolfin import *
from mpi4py import MPI

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import matplotlib.cm as cm

from optimal_distribution import *
from obsolete.gmsfem import *
from utils import get_simple_kappa

In [ ]:
#%load_ext line_profiler

In [2]:
def old_basis(Nv_on, xi, V, W):
    buf = []
    for v_dofs in Nv_on:
        v = Function(V)
        v.vector().set_local(v_dofs)
        
        w = Function(W)
        LagrangeInterpolator.interpolate(w, project(xi*v, V))
        buf.append(w)
    
    return buf

def new_basis(Nv_on, xi, V, W):
    buf = []
    for v_dofs in Nv_on:
        v = Function(V)
        v.vector().set_local(v_dofs)
        v.vector()[:] *= xi.vector()[:]
        
        w = Function(W)
        LagrangeInterpolator.interpolate(w, v)
        buf.append(w)
    
    return buf, Nv_on * xi.vector()[:]

def to_dofs(Psi_ms):
    buf = []
    for v in Psi_ms:
        buf.append(v.vector().get_local())
    return np.array(buf).reshape(N_cnbh, M_off, -1)

def resotre_fns(Nv, W):
    buf = []
    for dofs in Nv:
        w = Function(W)
        w.vector()[:] = dofs
        buf.append(w)
        
    return np.array(buf)

def get_coord(reg_id):
    col = reg_id % (n_blocks-1)
    row = reg_id // (n_blocks-1)
    return col, row
    
def build_V(reg_id):
    col, row = get_coord(reg_id)
    mesh = UnitSquareMesh(2*n_el, 2*n_el)
    mesh.translate(Point(col*.5, row*.5))
    mesh.scale(2./n_blocks)

    return FunctionSpace(mesh, 'P', 1)

def middle_basis(Nv_on, xi, W):
    V = xi.function_space()
    buf = []
    Nv_ms = []
    for v_dofs in Nv_on:
        v = Function(V)
        v.vector().set_local(v_dofs)
        v = project(xi*v, V)
        Nv_ms.append(v.vector()[:])
        
        w = Function(W)
        LagrangeInterpolator.interpolate(w, v)
        buf.append(w)
        
    return buf, np.array(Nv_ms)

def get_slice(A, i, j):
    return A[i*M_off:(i+1)*M_off, j*M_off:(j+1)*M_off]

# P1 Case

In [ ]:
from p1case import *

n_el = 8
n_blocks = 8 # must be > 3

N_el = n_el * n_blocks
N_cnbh = (n_blocks-1)**2

M_off = 10
M_on = 5
eta = 1e3
# -----------------

with open('cutils/pymodule2.cpp', 'r') as fp:
    cutils = compile_cpp_code(fp.read(), include_dirs=['cutils'])
K = get_simple_kappa(eta, N_el, seed=123)
W = K.function_space()
fine_mesh = W.mesh()

RHS = Expression('sin(2*pi*x[0])*sin(2*pi*x[1])', pi=pi, degree=1)
pairs = overlap_map(n_blocks-1)

cores = [None] * (n_blocks-1)*(n_blocks-1)
K_list, RHS_list = [], []

Nv_ms = np.empty((len(cores), M_off, (2*n_el+1)**2))
ms_dofs = np.empty((len(cores), M_off, ((N_el+1)**2)))

In [ ]:
%%time
for i in range(len(cores)):
    V = build_V(i)
    kappa = project(K, V)
    K_list.append(kappa)
    RHS_list.append(project(RHS, V))

    cores[i] = GMsFEUnit(i, n_el, n_blocks, cutils)
    Nv = cores[i].snapshotSpace(kappa)
    
    Nv,_= cores[i].modelReduction(kappa, Nv, M_off)
    Nv_ms[i] = cores[i].multiscaleFunctions(kappa, Nv)
    ms_dofs[i] = zero_extrapolate(Nv_ms[i], V, W, *get_coord(i))

In [ ]:
%%time
A = np.zeros((len(cores)*M_off, len(cores)*M_off))
b = np.zeros(len(cores)*M_off)

for i in range(len(cores)):
    A_ii, b_i = cores[i].diagonalBlock(K_list[i], Nv_ms[i], RHS_list[i])
    A[i*M_off:(i+1)*M_off, i*M_off:(i+1)*M_off] = A_ii
    b[i*M_off:(i+1)*M_off] = b_i
    
for i,j in pairs:
    A_ij = cores[i].offdiagonalBlock(K_list[i], Nv_ms[i], Nv_ms[j], j-i)
    A[i*M_off:(i+1)*M_off, j*M_off:(j+1)*M_off] = A_ij

In [ ]:
i_lower = np.tril_indices(len(A), -1)
A[i_lower] = A.T[i_lower]

sol = Function(W)
u = scipy.linalg.solve(A, b, assume_a='pos')
sol.vector().set_local(u @ ms_dofs.reshape(-1, W.dim()))

In [ ]:
%%time
v1 = TrialFunction(W)
v2 = TestFunction(W)
dx = Measure('dx', W)
a = K*dot(grad(v1), grad(v2)) * dx
L = RHS*v2*dx
u_f = Function(W)
bc = DirichletBC(W, Constant(0.), lambda x,on : on)

solve(a==L, u_f, bc)

In [ ]:
plt.figure(figsize=(32, 32))
plt.subplot(121)
p = plot(sol);
#plt.colorbar(p);

plt.subplot(122)
plot(u_f);

print(norm(project(u_f - sol, W)))

# General Case

In [ ]:
from core_components import *

n_el = 16
n_blocks = 8 # must be > 3

N_el = n_el * n_blocks
N_cnbh = (n_blocks-1)**2

M_off = 10
M_on = 5
eta = 1e3
# -----------------

with open('cutils/pymodule2.cpp', 'r') as fp:
    cutils = compile_cpp_code(fp.read(), include_dirs=['cutils'])
K = get_simple_kappa(eta, N_el, seed=123)
W = K.function_space()
fine_mesh = W.mesh()

rhs = 'sin(2*pi*x[0])*sin(2*pi*x[1])'
RHS = Expression(rhs, pi=math.pi, degree=1)

cores = [None] * (n_blocks-1)*(n_blocks-1) 
pairs = overlap_map(n_blocks-1)
Psi_ms = [None] * len(cores)
K_list, RHS_list = [], []

In [ ]:
%%time
for reg_id in range(len(cores)):
    V = build_V(reg_id)
    kappa = project(K, V)
    K_list.append(kappa)
    RHS_list.append(project(RHS, V))

    cores[reg_id] = GMsFEUnit(reg_id, n_el, n_blocks, cutils)
    Nv = cores[reg_id].snapshotSpace(kappa)
    Nv_on, w_off = cores[reg_id].modelReduction(kappa, Nv, M_off)
    xi = cores[reg_id].partitionFunction(kappa)
    Psi_ms[reg_id] = old_basis(Nv_on, xi, V, W)

Psi_ms = np.array(Psi_ms).flatten()
Nv_ms = to_dofs(Psi_ms)

In [ ]:
%%time
A = np.zeros((len(cores)*M_off, len(cores)*M_off))
b = np.zeros(len(cores)*M_off)

rhs_ = project(RHS, W)
for i in range(len(cores)):
    A_ii, b_i = cores[i].diagonalBlock(K, Nv_ms[i].tolist(), rhs_)
    A[i*M_off:(i+1)*M_off, i*M_off:(i+1)*M_off], b[i*M_off:(i+1)*M_off] = A_ii, b_i

In [ ]:
%%time
for i,j in pairs:
    pos = j - i
    A_ij = cores[i].offdiagonalBlock(K, Nv_ms[i].tolist(), Nv_ms[j].tolist(), pos)
    A[i*M_off:(i+1)*M_off, j*M_off:(j+1)*M_off] = A_ij

In [ ]:
i_lower = np.tril_indices(len(A), -1)
A[i_lower] = A.T[i_lower]

u = scipy.linalg.solve(A, b)
sol = project((u * Psi_ms).sum(), W)

In [ ]:
%%time
v1 = TrialFunction(W)
v2 = TestFunction(W)
dx = Measure('dx', W)
a = K*dot(grad(v1), grad(v2)) * dx
L = RHS*v2*dx
u_f = Function(W)
bc = DirichletBC(W, Constant(0.), lambda x,on : on)

solve(a==L, u_f, bc)

In [ ]:
plt.figure(figsize=(16, 16))
plt.subplot(121)
plot(sol);

plt.subplot(122)
plot(u_f);

print(norm(project(u_f - sol, W)))

# Colored Partition

In [ ]:
cp = ColoredPartition(9, 4)

In [ ]:
cp.partition(9)

In [ ]:
print((cp.map['c'] != None).sum())
print(cp.map['c'])
print()
print(cp.map['r'])

In [ ]:
A = 100*np.random.rand(1000, 100, 100)
B = A.astype('i8')
C = A.astype('i2')

%timeit C@C
%timeit B@B

# Python CPP module

In [ ]:
with open('cutils/pymodule2.cpp', 'r') as fp:
    cpp_code = fp.read()
    
cutils = compile_cpp_code(cpp_code, include_dirs=['cutils'])

In [ ]:
# MINIMAL WORKING EXAMPLE
import time
import numpy as np

from ufl import *
from dolfin import *

N = 32
M = 4*N
DIM = (N+1)*(N+1)
mesh = UnitSquareMesh(N, N)
V = FunctionSpace(mesh, 'P', 1)

Psi_array = []
kappa = Function(V)
kappa.vector()[:] = 1.

# Create simple functions for easier debugging
# In this case, Mass_matrix_ij = i*j
#               Stiffness_matrix_ij = 0
for i in range(M):
    psi = Function(V)
    psi.vector()[:] = i
    Psi_array.append(psi)

In [ ]:
Mass_matrix = np.empty((M, M))
elapsed_time =- time.time()
#----
for i in range(M):
    for j in range(i, M):
        Mass_matrix[i, j] = assemble(kappa*Psi_array[i]*Psi_array[j]*dx)
#----
elapsed_time += time.time()
print(f'Calculating mass matrix has taken {elapsed_time:.2f} s')

# Don't take into account transpose operation
ij_lower = np.tril_indices(M, -1)
Mass_matrix[ij_lower] = Mass_matrix.T[ij_lower]

Stiffness_matrix = np.empty((M, M))
elapsed_time =- time.time()
#----
for i in range(M):
    for j in range(i, M):
        Stiffness_matrix[i, j] = assemble(
            kappa*dot(grad(Psi_array[i]), grad(Psi_array[j]))*dx)
#----
elapsed_time += time.time()
print(f'Calculating stiffness matrix has taken {elapsed_time:.2f} s')

# Don't take into account transpose operation
ij_lower = np.tril_indices(M, -1)
Stiffness_matrix[ij_lower] = Stiffness_matrix.T[ij_lower]

In [ ]:
with open('cutils/nested_lists.cpp', 'r') as fp:
    cpp_code1 = fp.read()

with open('cutils/1d_arrays.cpp', 'r') as fp:
    cpp_code2 = fp.read()

with open('cutils/numpy_arrays.cpp', 'r') as fp:
    cpp_code3 = fp.read()
    
with open('cutils/pymodule.cpp', 'r') as fp:
    cpp_code4 = fp.read()

In [ ]:
cutils1 = compile_cpp_code(cpp_code1, include_dirs=['cutils']) # It seems like 
cutils2 = compile_cpp_code(cpp_code2, include_dirs=['cutils']) # optimization flags
cutils3 = compile_cpp_code(cpp_code3, include_dirs=['cutils']) # do not work.
cutils4 = compile_cpp_code(cpp_code4, include_dirs=['cutils']) # As well as any others

In [ ]:
CPsi_array = [x.cpp_object() for x in Psi_array]
stiffness = cutils4.stiffness_integral_matrix(kappa.cpp_object(), CPsi_array)
ij_lower = np.tril_indices(M, -1)
stiffness[ij_lower] = stiffness.T[ij_lower]
np.allclose(Stiffness_matrix, stiffness)

In [ ]:
CPsi_array = [x.cpp_object() for x in Psi_array]
mass = cutils4.mass_integral_matrix(kappa.cpp_object(), CPsi_array)
ij_lower = np.tril_indices(M, -1)
mass[ij_lower] = mass.T[ij_lower]
np.allclose(Mass_matrix, mass)

In [ ]:
def fn1():
    v2d = vertex_to_dof_map(V)
    F = np.empty((M, DIM), 'f8') 
    for i, psi in enumerate(Psi_array):
        F[i] = psi.vector()[v2d]
    K = kappa.vector()[v2d]
    mass = cutils1.build_mass_matrix(F, K)
    mass = np.array(mass)
    ij_lower = np.tril_indices(M, -1)
    mass[ij_lower] = mass.T[ij_lower]

def fn2():
    v2d = vertex_to_dof_map(V)
    F = np.empty((M, DIM), 'f8') 
    for i, psi in enumerate(Psi_array):
        F[i] = psi.vector()[v2d]
    K = kappa.vector()[v2d]
    data = np.r_[F.reshape(-1), K]
    mass = cutils2.build_mass_matrix(data)
    mass = mass.reshape(M, M)
    ij_lower = np.tril_indices(M, -1)
    mass[ij_lower] = mass.T[ij_lower]

def fn3():
    v2d = vertex_to_dof_map(V)
    F = np.empty((M, DIM), 'f8') 
    for i, psi in enumerate(Psi_array):
        F[i] = psi.vector()[v2d]
    K = kappa.vector()[v2d]
    mass = cutils3.build_mass_matrix(F, K)
    ij_lower = np.tril_indices(M, -1)
    mass[ij_lower] = mass.T[ij_lower]

def fn4():
    CPsi_array = [x.cpp_object() for x in Psi_array]
    mass = cutils4.mass_integral_matrix(kappa.cpp_object(), CPsi_array)
    ij_lower = np.tril_indices(M, -1)
    mass[ij_lower] = mass.T[ij_lower]

In [ ]:
t1 = %timeit -o fn1()
t2 = %timeit -o fn2()
t3 = %timeit -o fn3()
t4 = %timeit -o fn4()